# Ollama + OpenAI + Python

## 1. 모델 이름 지정하기

"phi3:mini"가 아닌 다른 모델을 가져왔다면, 아래 셀의 값을 변경하세요.  
이 변수는 노트북 전체에서 코드에 사용됩니다.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Open AI 클라이언트 설정

일반적으로 OpenAI 클라이언트는 OpenAI.com 또는 Azure OpenAI와 함께 사용되어 대규모 언어 모델과 상호작용합니다.  
하지만 Ollama와도 사용할 수 있습니다. Ollama는 "http://localhost:11434/v1"에서 OpenAI 호환 엔드포인트를 제공합니다.


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. 대화 응답 생성하기

이제 OpenAI SDK를 사용하여 대화에 대한 응답을 생성할 수 있습니다. 이 요청은 고양이에 관한 하이쿠를 생성해야 합니다:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. 프롬프트 엔지니어링

언어 모델에 처음으로 보내는 메시지는 "시스템 메시지" 또는 "시스템 프롬프트"라고 불리며, 모델의 전반적인 지침을 설정합니다.  
사용자는 자신만의 맞춤형 시스템 프롬프트를 제공하여 언어 모델이 다른 방식으로 출력을 생성하도록 유도할 수 있습니다.  
아래의 `SYSTEM_MESSAGE`를 수정하여 좋아하는 유명 영화/TV 캐릭터처럼 답변하도록 설정하거나, [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts)에서 다른 시스템 프롬프트에 대한 영감을 얻을 수 있습니다.

시스템 메시지를 사용자 정의한 후, `USER_MESSAGE`에 첫 번째 사용자 질문을 입력하세요.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. 몇 가지 예시

언어 모델을 안내하는 또 다른 방법은 "몇 가지 예시"를 제공하는 것입니다. 이는 질문/답변의 예시를 순서대로 제시하여 모델이 어떻게 반응해야 하는지를 보여주는 방식입니다.

아래 예시는 언어 모델이 조교처럼 행동하도록 유도하는 방법을 보여줍니다. 조교가 할 법한 질문과 답변 몇 가지를 제공한 후, 학생이 할 법한 질문으로 모델을 유도합니다.

먼저 시도해보고, 새로운 상황에 맞게 `SYSTEM_MESSAGE`, `EXAMPLES`, 그리고 `USER_MESSAGE`를 수정해보세요.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. 검색 증강 생성

RAG (검색 증강 생성)은 특정 도메인에 대해 언어 모델이 질문에 정확히 답변하도록 하는 기술로, 먼저 지식 소스에서 관련 정보를 검색한 후 해당 정보를 기반으로 응답을 생성합니다.

하이브리드 자동차에 대한 데이터를 포함한 로컬 CSV 파일을 제공했습니다. 아래 코드는 CSV 파일을 읽고 사용자 질문과 일치하는 내용을 검색한 후, 발견된 정보를 기반으로 응답을 생성합니다. 이전 예제들보다 시간이 더 걸릴 수 있다는 점을 유의하세요. 이는 모델에 더 많은 데이터를 보내기 때문입니다. 만약 응답이 여전히 데이터에 기반하지 않는다고 느껴진다면, 시스템 엔지니어링을 시도하거나 다른 모델을 사용해볼 수 있습니다. 일반적으로 RAG는 더 큰 모델이나 SLM의 미세 조정된 버전과 함께 사용할 때 더 효과적입니다.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역을 사용함으로써 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
